In [1]:
import os
import numpy as np
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4041 --packages org.apache.kafka:kafka_2.11:0.9.0.0,org.apache.kafka:kafka-clients:0.9.0.0  pyspark-shell'

In [2]:
from pyspark import SparkContext
sc = SparkContext("local[1]", "KafkaSendStream") 
from kafka import KafkaProducer
import time

In [5]:
import json
from satori.rtm.client import make_client, SubscriptionMode
from math import log

# Make a proxy class

endpoint = "wss://open-data.api.satori.com"
appkey = "dC6c33Fbb5ECdAC1Ef2aB77dcBfBB0B0"
channel = "cryptocurrency-market-data"

with make_client(endpoint=endpoint, appkey=appkey) as client:
    print('Connected to Satori RTM!')
    messagebox=[]

    class SubscriptionObserver(object):
        def on_subscription_data(self, data):
            for message in data['messages']:
                messagebox.append(message)

    subscription_observer = SubscriptionObserver()
    client.subscribe(
        channel,
        SubscriptionMode.SIMPLE,
        subscription_observer,
        {'filter':'select * from `cryptocurrency-market-data` where exchange = "Bitstamp" and basecurrency = "USD" and cryptocurrency="BTC"'})
        #  exchange != "ShapheShift" OU type != "ratequote" si on laisse ShapeShift

    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    i=0
    n=2   # number of inputs
    numOfRendements=20
    """
    while not messagebox: # waiting for a first price to arrive
        time.sleep(1)
    lastPrice=float(json.loads(json.dumps(messagebox.pop(), ensure_ascii=False))['price']) # get that first price
    while True:
        proxies=np.empty(0) # x of the RLS algorithm
        while proxies.size<n+1:
            rendements=np.empty(0) 
            while rendements.size<numOfRendements: # Choix du nombre de rendements pour le calcul du proxy            
                if messagebox:
                    message=messagebox.pop()
                    message=json.dumps(message, ensure_ascii=False)
                    message=json.loads(message)
                    rendement=np.log(float(message['price'])/lastPrice) # r = np.log(float(message['price'])/lastPrice)
                    #print(float(message['price']), lastPrice, rendement)
                    lastPrice=float(message['price'])
                    rendements=np.append(rendements, rendement)
                    #print(float(message['price']), lastPrice, log(float(message['price'])/lastPrice))print(rendements, rendements.size)
                time.sleep(1)
            proxies=np.append(proxies, np.sum(np.square(rendements))/numOfRendements)
    """
    
    """
        Scikit results:

        [[-0.02496175 -0.04879209]]
        Mean squared error: 0.93
        Variance score: 0.01

    """
    
    X_train=([-0.14422805108138562, 2.083771292103893], [0.02368754116177894, 0.3407276600907841], [-0.14629314255353457, 0.5132683953894979], [-0.48928034996323794, 0.1562093992134668], [0.9072952920262846, 0.7306827038921886], [-0.10404602589165644, 0.6714717090946938], [-0.09272123353418273, 0.5255259976163705], [0.18997763564544068, 0.29814680781437164], [-1.423099250615641, 1.0055560375118802], [-0.30434180856331844, 0.9419732205130641], [1.7829655053792985, -0.22047326748499815], [-0.378392594295389, 0.8586411181781818], [-0.7308321550884557, -0.791526322701571], [-1.1278448448428888, 0.14457843973453607], [-0.19268912421065093, 0.4367967173566577], [-0.09040903011267723, 1.902334006869799], [0.011015874289171201, -2.3494297962330535], [-0.8637754737215552, -0.25808869667529066], [-0.6504665184989179, -0.5601054695788505], [0.5759721283102504, -0.09797455909101191], [-2.209773097851861, -0.3485613555708654], [0.6367093658522285, -1.141646102569762], [0.33024708847948836, 2.220494587325087], [0.5762904061229949, 0.33604781383303106], [1.7013427697416268, -0.19412307673649865], [-0.5510306539688685, 0.273105118841396], [-1.8850787314691648, 0.38620734639145604], [-0.8935104956918535, -1.694216945382065], [-0.23447696255854203, 2.555965080077589], [-0.7095720737253641, 0.7410072646661071], [-0.9372188951382652, -2.269630943528133], [0.2334179977460624, 0.019068311719007597], [0.868847064726637, 1.0898809570845374], [-3.329546236561662, 0.7955168408861591], [-0.1337504517298234, 1.8521108252257967], [2.1448781134760515, -1.0786097861864072], [0.4516228866144898, -0.23427030072986596], [1.88746425351911, 0.6310894627324672], [-1.7857580168172265, -0.2837227077137797], [0.8730284056231602, -1.4624095988851413], [1.2071976478689659, -0.21489694717395244], [-0.6611833652848388, -0.24783558656383198], [1.0659107327623993, -0.570121058868149], [-0.30130470163048323, 1.024008833355122], [-0.2702100205017047, -1.1097550505936047], [1.3585585376331044, -1.1796124492217945], [-0.4295077167842404, -0.9629553086583236], [0.9255684468090027, -1.0897213536719965], [0.25845254758191666, 0.25739594379765474], [0.55840809161697, -1.26406206056951], [0.5076447955872034, 0.36272395535044216], [0.8718219007100112, -0.035120746882075025], [-0.15594343377857206, -0.40973713623660785], [-1.1273874058332376, -0.9038645352866782], [-0.35520310635887203, -0.03413538881195157], [-0.0027871040267202886, -0.5308420397449], [1.4129332494240028, -0.5665248075577987], [0.9922722470170022, -0.845196157924356], [1.4335877371715657, -0.9617095767648888], [-2.755146411801248, 0.09749453451029202], [0.5090601007790747, 0.9684608272258519], [1.0989817482404385, -0.43347708193571577], [-0.9074495196385102, -0.08027838531499759], [0.4485327885492952, 0.2938942363827525], [-0.9545700960735355, 1.1091687929826968], [-0.3294482649440561, 1.1172219036897835], [0.4523185399860011, 0.18139163023151972], [0.3974919003387145, -1.6552343208425984], [0.11063415664907739, 0.17763185955946453], [1.2533085348726831, -1.2520527278839426], [0.22961481443566079, 0.05474109734634394], [-0.6391213233333362, 1.3275359337908317], [0.27454239586220247, 0.7264306898971987], [1.8611420782445587, -0.9745099984790421], [-0.11952631331926476, -0.6586194817470897], [-1.344252654740801, -0.13859156576072476], [0.8460190684337479, 0.029604775416872023], [-0.06033165691861222, 0.9273633463432431], [-0.14666543409817823, 1.4150590294440675], [1.0555242371536298, 0.06770057144475106], [-1.0087001363670252, 0.12171943281948051], [-0.02712149881887855, -1.4905095185719988], [-0.7451186965042212, -2.014927130231162], [1.142045650750487, 1.0432139029525767], [1.271683962411291, -0.12210272132584393], [1.6535349328566704, 1.4978682719975973], [-0.1838173029541714, -1.5250001734468257], [0.24938451113115234, 1.004377140765803], [-0.016120900260351338, 0.17078723708942017], [-1.3513916520278166, 0.431608358247296], [-0.6077946510230444, 1.6845617816337797], [1.366951546389567, -0.12624146361208816], [0.49790501252904734, -1.257833326653457], [-0.7716189805510948, 0.9939651951237896], [1.014540015735707, 0.8122330824420447], [-1.2870627950880287, 0.21030873672988], [-0.9793183295971389, -0.42456370480389327], [-0.8898934869709499, -1.4575354336486575], [-0.9557878519069367, 2.5008719278664437], [2.189108198779184, -1.2364162629373283])
    X_test=([1.1266188489898357, -0.34100220021144456], [0.7059132451445839, 0.08690474534397866], [1.4661114150754222, -1.4193306944057373], [-2.109467209716234, 0.20488813574550738], [-0.7564690774014791, -0.14237016789642726], [1.7270777436464002, -0.42788802814926], [-1.7207013638919277, -1.5067063142791746], [1.4560535375835246, 0.8653499271069528], [0.15733293065807744, 0.933538375680115], [-1.5535896495572534, -0.25784103726115665], [-1.6895222973736623, -0.25125158287134014], [2.146018704275312, -0.5438632469203741], [0.3909537468333275, 0.7589785926077027], [-0.6533767781886657, -0.037736165811227004], [-0.02943101362451333, 0.10876094790889583], [0.4591992322334427, 0.5418024176172381], [-1.314595068016085, -1.7838938143144318], [1.507641960813707, 0.07117156855794741], [-1.2926612184568134, 0.6142272795427536], [-1.4991897687616662, 1.4813685992639767], [2.061801716876898, -1.301727383813708], [0.9274806920479677, -0.8765123307085451], [0.11743614482229775, -1.9159801938790137], [-0.3875425295487655, 1.501931956158538], [0.5958375775013759, 1.3936319685926055], [1.7826048296627177, 1.1022669433704506], [-0.08520018295745276, 0.8698508791974099], [-1.5110968875019357, 0.4283782531997773], [-0.774570816946808, -2.238757830671146], [-0.19710217172195724, -1.0915715352908084], [0.9013291812569912, 2.379068924693173], [-0.20319995423621118, -0.7144905529833843], [-0.2795957752562504, 0.9374424826367157], [-0.14710723649111884, 1.9551411875661837], [1.0905768316012558, 1.9820940779576957], [1.0563736168498787, -0.06243253757044173], [0.2556217403437049, 1.2350754542949913], [0.2885280625362472, -0.5167628410927732], [-1.6826272405750284, 0.4188661248081167], [0.7561907852539256, 1.5143149258238622], [-0.3982031276906908, -1.5634334976196904], [1.3847141386319708, 0.3525788074018525], [2.184967020396703, -0.22224871792222983], [0.2735506214766027, 0.17375812039024047], [0.33707165369828745, 0.6598604394965403], [-0.49881599625071366, -0.8786605588066674], [-0.03278200107526172, -1.461424075066885], [0.6709030917379651, -0.05550917221135103], [-1.3449093816330413, 1.2086625428124416], [-0.3176926307426424, 0.20183280662289496], [0.08012242160476075, 0.9509115789816353], [3.1564363654712713, -1.1318440812097617], [-2.3224133182644087, -0.5093593181377696], [0.3604265953123225, -0.02066216903478962], [1.5445083044327763, 0.03264419059013152], [0.9890592463438145, 0.7233207791553532], [1.9143618269390814, 1.2552861972765101], [-0.694858832203385, -0.527378021694278], [0.6105810116202265, -1.577833885996817], [0.3351470865869174, -1.002932665692971], [2.030497495632751, -0.5239721593058295], [0.17065365979959035, -0.15477508481502425], [1.2836976206679473, 1.8895937118484065], [0.4173048042553628, 0.576455717446262], [-1.8924553221979579, 0.2137116413350416], [0.39500741508946907, 0.08313726587498252], [-1.0236991835328495, 0.015630009441750654], [0.9430788499109622, 0.3932805211425225], [-0.03838397275911862, 1.4615261960181802], [-1.4746680006422472, 1.1720132425483782], [-1.0752083748651842, -0.876669076878471], [0.5661532359746225, 2.3732238576394336], [0.25223542973828483, 1.032803272257297], [-1.4014473849001106, -0.30511128860382625], [-0.21898065358812915, -0.09061112886427858], [0.7206671411858578, -0.848987192568518], [0.5664259207552534, -1.4126478227223986], [1.426015517082363, -1.0170809845198563], [-0.4211214468437763, 0.7168361843129163], [1.76960311850783, 0.2411077809650336], [-0.38849313760051124, 1.1978971279209119], [0.9921863847090969, 0.7865000292967447], [1.6184194133057568, -0.45131981383452385], [0.7814990417929647, 0.3417608260026375], [-0.02877484292218688, 0.3523144699982993], [0.8843598787351217, -0.4599434412594454], [-1.4521234987985856, -1.170532394675803], [-0.557897633259946, 1.5006760596408544], [-0.12552396243440367, -0.16330876475210176], [-1.5422520861939457, 0.9151945426208387], [1.2780446343702285, 0.08529019467956919], [2.1008272908550336, -0.10335564985560655], [-0.3940897830548165, 2.9672912639897624], [0.41286452157123865, 0.8168590507630964], [-0.3995942798828767, -0.027747280790187324], [2.1679127625125245, -1.1473901339067818], [2.0993718314074554, -1.3657053175512508], [-0.673984444585695, 1.3104502751567288], [0.4328730272017391, 0.7964190920857935], [-0.23692989418485946, -1.05320035965511])
    Y_train=([0.4563104534443858], [-0.770940458256527], [-0.1161718869102406], [1.6594625448225135], [-0.32221440105101085], [0.6194990567843599], [1.0588361173449163], [-1.4004423651398832], [1.776864260598746], [0.24071253108592158], [0.30362227354987104], [1.2251530143779716], [0.9563718237256785], [-0.8395821943620106], [1.331578983876477], [1.1736487825317046], [0.004609640861253766], [2.330460693710105], [-0.6578383299928034], [0.4064880884493528], [-1.7763075857524273], [-1.1902690154207], [-1.1538648933982354], [-1.7265610435727838], [0.4748365802425942], [1.9847693995712903], [-0.14576193226744555], [0.37492532945591156], [-1.3110117560800054], [0.3445898372670604], [-1.8662296860875522], [1.0416090802744897], [-1.4775687108640836], [0.6804052263692578], [-0.33151861519543413], [-0.5190918398454057], [-1.4500355196759094], [0.5476054837783054], [-1.0406666854490894], [1.1976602809243437], [-0.23370191725539366], [2.4955688939862153], [2.028397967099057], [1.1703839098616189], [-0.47861664850770674], [0.41486806516400215], [-0.5804364882757246], [0.4656595073994698], [-0.6195891051753725], [-0.21292270888785947], [-1.7192031300196717], [-0.16919525777910083], [-1.8418741646768757], [-1.4948332706639464], [0.4117296782391435], [1.3005694240956573], [-1.2075797853430321], [-0.2596798034403577], [0.3891241367429729], [0.44993168745505047], [0.22920007894495645], [-1.0363224934127873], [-0.20266148931153022], [0.6962309817051311], [-0.3360584276713208], [-0.02819884347200902], [-0.854143408243279], [-1.0217945144094878], [-0.030279284966717605], [-0.057836849829224976], [0.09596906198913738], [0.11331443924167468], [-0.2620162401718843], [-0.6332567535189053], [-0.334449425898562], [0.7368322980545744], [-0.9144994539850002], [-1.531720001773797], [-0.404716426367039], [-0.33124751052830775], [-0.7489887582578352], [-0.5096063255644917], [-0.4275993712077629], [0.8524808828124218], [-0.7495212156273525], [0.5042059494909495], [1.0684828692387272], [-0.21230516442881872], [-1.319641852761248], [0.2514415594945603], [-0.9118926841683095], [0.2599297194065436], [-0.5032179990172394], [-0.5430498411145288], [0.29551746925359174], [-1.1319004799926335], [1.5820207902931214], [1.7097690575549356], [-2.055298897119167], [1.5727072793774135])
    Y_test=([-0.956783439488732], [-0.12696739688477543], [-1.4163225545458935], [1.100473997147404], [0.8145808132481821], [0.626253081476264], [0.09013961225107298], [0.5770660852802297], [-0.44542630229684593], [1.3428916835874485], [-0.420544120421918], [0.2852005185412514], [-0.6220229174440771], [1.0406265545149294], [0.6098586414130723], [-0.41498075010410324], [1.1290549133655228], [0.8083236283274108], [1.1453758933537026], [0.3120951176722594], [0.49175277003118595], [1.24531705134753], [-0.14615903472693323], [-0.1845779851932924], [1.8057971083988713], [-0.6151941922798337], [-1.4508354337748386], [-1.4273168883538914], [-0.3581593733988018], [-0.8546192724837812], [0.5562686073647954], [0.4757269860391185], [-0.23442938320643072], [-0.05141203345438175], [-1.2496628964669572], [-0.880439981241259], [0.44114628502247283], [-1.5461782903081625], [-0.013029432300442153], [-0.5825483057920186], [-0.5080538686344003], [0.9777601803924558], [-0.5465747527539397], [-1.9703522057477565], [-0.6949750299272058], [-0.5750735109582203], [0.951177131400584], [1.2460097018570147], [0.26624131174890486], [-1.0614817182061294], [1.9228919457655613], [-1.928108841806785], [1.1620339431093352], [0.327338040175728], [-0.562430936952977], [0.08962590468017492], [-0.6119342705251675], [-0.25244701675872994], [-0.4965789845030086], [-0.5772023434194452], [0.24541850327267412], [0.8977984001566954], [-0.2570811571779318], [1.0445080506261166], [-0.7635846845935493], [0.5646104213083958], [0.38144197975112254], [-0.2648083965169878], [-2.101441701239086], [-0.32914445476441256], [3.0048642848063145], [-0.07654756539480691], [-0.4658924525582484], [-0.16938005797474798], [-0.3414826028920431], [-0.1328062601602184], [0.8999590117359733], [-0.7468890266650675], [-1.7415628380806243], [-0.8585133349316486], [-0.5501216936365939], [-1.133776318210735], [-0.34401237827974523], [0.08053069647234688], [0.10003386623886339], [0.9804299636945164], [0.4834569971928504], [1.1798284108552082], [0.7812875929328751], [-1.4258480539350222], [0.09974249773987462], [-1.4729607071423492], [-0.6150355882120132], [0.19491118096252336], [-0.9018423463668116], [-1.4846481354072045], [0.5183215561901968], [-0.9526501967754473], [2.233193480252188], [-1.9222711652093982])

    for x in range(100):
        proxies=np.array([X_train[x][0], X_train[x][1]])
        y=Y_train[x][0]
        message=np.array2string(np.append([i,y], proxies),separator=",") 
        producer.send('test2', message)
        i=i+1
        print(message)
    for x in range(100):
        proxies=np.array([X_test[x][0], X_test[x][1]])
        y=Y_test[x][0]
        message=np.array2string(np.append([i,y], proxies),separator=",") 
        producer.send('test2', message)
        i=i+1
        print(message)
    """
        while True:
            proxies=np.roll(proxies,-1)
            proxies[0]=1
            while not messagebox:
                time.sleep(1)
            message=messagebox.pop()
            message=json.dumps(message, ensure_ascii=False)
            message=json.loads(message)
            proxy+=(np.log(float(message['price'])/lastPrice)**2)/j # r = np.log(float(message['price'])/lastPrice)
            lastPrice=float(message['price'])
            proxies[n-1]=proxy
            j+=1
            y=np.sum(proxies)+0.1*np.random.rand(1)[0]
            message=np.array2string(np.append([i,y], proxies),separator=",") 
            producer.send('test2', message)
            i=i+1
            print(message)
    """

Connected to Satori RTM!
[ 0.        , 0.45631045,-0.14422805, 2.08377129]
[ 1.        ,-0.77094046, 0.02368754, 0.34072766]
[ 2.        ,-0.11617189,-0.14629314, 0.5132684 ]
[ 3.        , 1.65946254,-0.48928035, 0.1562094 ]
[ 4.        ,-0.3222144 , 0.90729529, 0.7306827 ]
[ 5.        , 0.61949906,-0.10404603, 0.67147171]
[ 6.        , 1.05883612,-0.09272123, 0.525526  ]
[ 7.        ,-1.40044237, 0.18997764, 0.29814681]
[ 8.        , 1.77686426,-1.42309925, 1.00555604]
[ 9.        , 0.24071253,-0.30434181, 0.94197322]
[ 10.        ,  0.30362227,  1.78296551, -0.22047327]
[ 11.        ,  1.22515301, -0.37839259,  0.85864112]
[ 12.        ,  0.95637182, -0.73083216, -0.79152632]
[ 13.        , -0.83958219, -1.12784484,  0.14457844]
[ 14.        ,  1.33157898, -0.19268912,  0.43679672]
[ 15.        ,  1.17364878, -0.09040903,  1.90233401]
[  1.60000000e+01,  4.60964086e-03,  1.10158743e-02, -2.34942980e+00]
[ 17.        ,  2.33046069, -0.86377547, -0.2580887 ]
[ 18.        , -0.65783833,

[ 144.        ,  -0.69497503,   0.33707165,   0.65986044]
[ 145.        ,  -0.57507351,  -0.498816  ,  -0.87866056]
[  1.46000000e+02,  9.51177131e-01, -3.27820011e-02, -1.46142408e+00]
[  1.47000000e+02,  1.24600970e+00,  6.70903092e-01, -5.55091722e-02]
[ 148.        ,   0.26624131,  -1.34490938,   1.20866254]
[ 149.        ,  -1.06148172,  -0.31769263,   0.20183281]
[  1.50000000e+02,  1.92289195e+00,  8.01224216e-02,  9.50911579e-01]
[ 151.        ,  -1.92810884,   3.15643637,  -1.13184408]
[ 152.        ,   1.16203394,  -2.32241332,  -0.50935932]
[  1.53000000e+02,  3.27338040e-01,  3.60426595e-01, -2.06621690e-02]
[  1.54000000e+02, -5.62430937e-01,  1.54450830e+00,  3.26441906e-02]
[  1.55000000e+02,  8.96259047e-02,  9.89059246e-01,  7.23320779e-01]
[ 156.        ,  -0.61193427,   1.91436183,   1.2552862 ]
[ 157.        ,  -0.25244702,  -0.69485883,  -0.52737802]
[ 158.        ,  -0.49657898,   0.61058101,  -1.57783389]
[ 159.        ,  -0.57720234,   0.33514709,  -1.00293267]
